BBC - selenium

In [1]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
from datetime import datetime
import requests
import time
from bs4 import BeautifulSoup



In [2]:
columns = ['title', 'content', 'author', 'url' ,'date', 'year', 'month', 'day', 'regions', 'tags', 'summary', 'og_link', 'original_lang', 'label']
df_real= pd.DataFrame(columns = columns)
df_real = pd.read_csv("real_bbc.csv", index_col=0)
df_real

,title,content,author,url,date,year,month,day,regions,tags,summary,og_link,original_lang,label
0,"Ukraine war: Zelensky says 31,000 troops kille...","Volodymyr Zelensky says 31,000 Ukrainian soldi...",Kathryn Armstrong,https://bbc.com/news/world-europe-68397525,2024-02-25 00:00:00,2024.0,2.0,25.0,NaN,NaN,NaN,https://bbc.com/news/world-europe-68397525,en,0
1,All of Europe has 'skin in the game' in Ukrain...,,NaN,https://bbc.com/news/world-europe-68392036,NaN,0.0,0.0,0.0,NaN,NaN,NaN,https://bbc.com/news/world-europe-68392036,en,0
2,Ukrainian woman gives Welsh pub new lease of life,A woman who fled war-torn Ukraine hopes that r...,Andrew Arthur,https://bbc.com/news/articles/cj7vlvy1yvro,2024-03-05 00:00:00,2024.0,3.0,5.0,NaN,NaN,NaN,https://bbc.com/news/articles/cj7vlvy1yvro,en,0
3,Refugee returned to Kyiv due to state of UK home,A Ukrainian refugee returned to live in Kyiv r...,Gemma Dunstan,https://bbc.com/news/articles/c84jp1g2j71o,2024-03-14 00:00:00,2024.0,3.0,14.0,NaN,NaN,NaN,https://bbc.com/news/articles/c84jp1g2j71o,en,0
4,The unlikely species entangled in Ukraine's re...,"Lawsuits, art and the sandy blind mole-rat hav...",Darya Tsymbalyuk,https://bbc.com/future/article/20240223-ukrain...,2024-02-24 00:00:00,2024.0,2.0,24.0,NaN,NaN,NaN,https://bbc.com/future/article/20240223-ukrain...,en,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,Lockdown parties: PM accepts responsibility ov...,"Boris Johnson ""accepts responsibility"" for bre...",NaN,https://bbc.com/news/uk-politics-60925233,2022-03-30 00:00:00,2022.0,3.0,30.0,NaN,NaN,NaN,https://bbc.com/news/uk-politics-60925233,en,0
2272,Prayers for Ukraine as crowds join anti-war demo,Prayers have been said for a peaceful end to t...,NaN,https://bbc.com/news/uk-scotland-60543797,2022-02-27 00:00:00,2022.0,2.0,27.0,NaN,NaN,NaN,https://bbc.com/news/uk-scotland-60543797,en,0
2273,Ukraine war: Russian investigator says 92 Ukra...,Moscow has charged 92 members of the Ukrainian...,NaN,https://bbc.com/news/world-europe-62287502,2022-07-25 00:00:00,2022.0,7.0,25.0,NaN,NaN,NaN,https://bbc.com/news/world-europe-62287502,en,0
2274,Ukrainian ballet star Artem Datsyshyn dies aft...,Leading Ukrainian ballet dancer Artem Datsyshy...,NaN,https://bbc.com/news/entertainment-arts-60794419,NaN,0.0,0.0,0.0,NaN,NaN,NaN,https://bbc.com/news/entertainment-arts-60794419,en,0


In [37]:

def get_data(start_page):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    cookie = {
        'name': 'permutive-consent',
        'value': '{"opt_in":true,"token":"CMP_ENABLED_NOT_SIGNED_IN"}'
    }
    page = 0
    options = Options()
    options.add_argument("--headless")  # Ensures Chrome runs in headless mode
    options.add_argument("--no-sandbox")  # Bypass OS security model, REQUIRED on Linux
    options.add_argument("--disable-dev-shm-usage")
    #options.add_argument()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")


    driver = webdriver.Chrome(options=options) 
    wait = WebDriverWait(driver, 25)
    # Your web scraping or automation code
    driver.get("https://www.bbc.com/search?q=war%20in%20ukraine")

    driver.add_cookie(cookie)
    driver.refresh()

    df_real_temp = pd.DataFrame(columns=columns)
# Wait for dynamic content to load
    end_page = start_page + 50
    page = start_page
    if page != 1:
            for _ in range(page):
                try:
                    next_button =  WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="pagination-next-button"]')))
                
                    driver.execute_script("arguments[0].click();", next_button)

                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'[data-testid = liverpool-card]')))

               
                except TimeoutException:
                    print("Next page button not found or last page reached.")
                    break
    # Loop to navigate pages and scrape content
    for _ in range(start_page, end_page): 
        

        article_links = []
        print(page)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        try:
        
            liverpool_cards = soup.find_all('div', {'data-testid': 'liverpool-card'})
        except:
            print("Error getting the cards.")

        
        try:
        # Iterate over each liverpool-card div to extract the link
            for liverpool_card in liverpool_cards:
                # Find the anchor tag within the liverpool-card div
                anchor_tag = liverpool_card.find('a', {'data-testid': 'internal-link'})
                # Extract the href attribute value from the anchor tag
                if anchor_tag:
                    link = "https://bbc.com" + anchor_tag['href']
                    article_links.append(link)
                    print(link)
                
        except:
            print("Error getting article links.")

        try:
            for article in article_links:
            
                print(article)
                try:
                    driver.get(article)
                    print('Succesfully loaded')
                except:
                    continue
                title = ""
                try:
                    title_element =  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME,'h1')))
                    title = title_element.text
                    print(title)
                except Exception as e:
                    print(f"Error loading title: {e}")


                try:
                    date_element = driver.find_element(By.TAG_NAME, 'time')
                    if date_element:
                        date = date_element.text
                        if date == "1 day ago":
                            date="2024-03-19"
                        elif date =="2 days ago":
                            date="2024-03-18"
                        elif date =="3 days ago":
                            date="2024-03-17"
                        elif date =="4 days ago":
                            date="2024-03-16"
                        elif date =="5 days ago":
                            date="2024-03-15"
                        elif date =="6 days ago":
                            date="2024-03-14"
                        elif date =="7 days ago":
                            date="2024-03-13"

                        date = pd.to_datetime(date)
                    else:
                        date = "NA"
                    print(date)
                except:
                    print("No time")
                    date = 'NA'
                    
                author = ''
                try:
                    author_ = driver.find_element(By.CSS_SELECTOR, '[data-testid = byline-name]')
                    if author_:
                        author = author_.text
                        author = author.replace("By ", "").rstrip(", ")
                    else:
                        author = "Na"

                 
                except:
                    print("Error getting author")
                    

                text= ""
                try:
                    content = driver.find_elements(By.CSS_SELECTOR,'[data-component=text-block]')
                    if content:
                         text = " ".join(element.text for element in content)
                    else:
                        text=" "
                    
                except:
                    print('Error handling content.')
                
                try:
                    regions_text= ""
                    tags = ""
                    summary = ""
                    if date== "NA":
                        year = 0
                        day = 0
                        month = 0
                    else:
                        year = date.year
                        day = date.day
                        month = date.month
                    new_page_data = [title, text, author, article, date, year, month, day, regions_text, tags, summary, article, "en", 0]
                    print(f'Title: {title}, Author: {author}')
                    df_real_temp = df_real_temp.append(pd.Series(new_page_data, index = df_real_temp.columns), ignore_index = True)
                except Exception as e:
                    print(f'Error adding article to df:{e}')
                
        
        except:
            print("Error with article_links")
        #page navigation 
        driver.get("https://www.bbc.com/search?q=war%20in%20ukraine")
        for _ in range(page):
                try:
                    next_button =  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="pagination-next-button"]')))
                   
                    driver.execute_script("arguments[0].click();", next_button)

                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'[data-testid = liverpool-card]')))

                
                except TimeoutException:
                    print("Next page button not found or last page reached.")
                    break

        page +=1
        
        time.sleep(5)
    driver.quit()   
    return df_real_temp

        


In [38]:
df_real_temp = get_data(454) #1 -51 > 52 - 153

454
https://bbc.com/news/uk-60560589
https://bbc.com/news/business-60948439
https://bbc.com/news/world-europe-64038140
https://bbc.com/news/world-europe-60979305
https://bbc.com/sport/64221042
https://bbc.com/news/world-europe-43457694
https://bbc.com/news/uk-scotland-62479329
https://bbc.com/news/business-64303706
https://bbc.com/news/world-europe-65971790
https://bbc.com/news/uk-60560589
Succesfully loaded
Ukraine crisis: Ukrainians prepare to go home to fight
2022-02-28 00:00:00
Title: Ukraine crisis: Ukrainians prepare to go home to fight, Author: Tom Symonds
https://bbc.com/news/business-60948439


C:\Users\annak\AppData\Local\Temp\ipykernel_8540\1191623899.py:172: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_real_temp = df_real_temp.append(pd.Series(new_page_data, index = df_real_temp.columns), ignore_index = True)


Succesfully loaded
Diesel delivery highlights UK dependence on Russia
2022-04-03 00:00:00
Title: Diesel delivery highlights UK dependence on Russia, Author: Ben King
https://bbc.com/news/world-europe-64038140
Succesfully loaded
Ukraine war: Volodymyr Zelensky visits front-line city of Bakhmut
2022-12-20 00:00:00
Title: Ukraine war: Volodymyr Zelensky visits front-line city of Bakhmut, Author: Hugo Bachega in Kyiv and Robert Greenall in London
https://bbc.com/news/world-europe-60979305
Succesfully loaded
Moment Hungary PM criticises President Zelensky in winner's speech
No time
Error getting author
Title: Moment Hungary PM criticises President Zelensky in winner's speech, Author: 
https://bbc.com/sport/64221042
Succesfully loaded
Leisure services at risk without further support for energy bills, UK government warned
No time
Error getting author
Title: Leisure services at risk without further support for energy bills, UK government warned, Author: 
https://bbc.com/news/world-europe-43457

In [39]:
df_real_temp

,title,content,author,url,date,year,month,day,regions,tags,summary,og_link,original_lang,label
0,Ukraine crisis: Ukrainians prepare to go home ...,A growing number of Ukrainians living in Brita...,Tom Symonds,https://bbc.com/news/uk-60560589,2022-02-28 00:00:00,2022,2,28,,,,https://bbc.com/news/uk-60560589,en,0
1,Diesel delivery highlights UK dependence on Ru...,"On Saturday afternoon, a bright red tanker car...",Ben King,https://bbc.com/news/business-60948439,2022-04-03 00:00:00,2022,4,3,,,,https://bbc.com/news/business-60948439,en,0
2,Ukraine war: Volodymyr Zelensky visits front-l...,Ukrainian President Volodymyr Zelensky has mad...,Hugo Bachega in Kyiv and Robert Greenall in Lo...,https://bbc.com/news/world-europe-64038140,2022-12-20 00:00:00,2022,12,20,,,,https://bbc.com/news/world-europe-64038140,en,0
3,Moment Hungary PM criticises President Zelensk...,,,https://bbc.com/news/world-europe-60979305,NA,0,0,0,,,,https://bbc.com/news/world-europe-60979305,en,0
4,Leisure services at risk without further suppo...,,,https://bbc.com/sport/64221042,NA,0,0,0,,,,https://bbc.com/sport/64221042,en,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,War in Ukraine: A family's struggle to get to ...,"A Scottish woman says she feels a ""sense of gu...",,https://bbc.com/news/uk-scotland-61192113,2022-04-24 00:00:00,2022,4,24,,,,https://bbc.com/news/uk-scotland-61192113,en,0
445,Britons with Ukrainian family criticise visa b...,The difficulties faced by British families att...,,https://bbc.com/news/uk-england-northamptonshi...,2022-03-11 00:00:00,2022,3,11,,,,https://bbc.com/news/uk-england-northamptonshi...,en,0
446,Oleksandr Zinchenko: Ukraine & Man City defend...,,,https://bbc.com/sport/av/football/60627422,NaN,NaN,NaN,NaN,,,,https://bbc.com/sport/av/football/60627422,en,0
447,Shells rain down on Kharkiv as Ukraine's army ...,Ukraine's second city Kharkiv has been the con...,,https://bbc.com/news/world-europe-60785679,2022-03-17 00:00:00,2022,3,17,,,,https://bbc.com/news/world-europe-60785679,en,0


In [31]:
df_real_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          898 non-null    object
 1   content        898 non-null    object
 2   author         898 non-null    object
 3   url            898 non-null    object
 4   date           897 non-null    object
 5   year           897 non-null    object
 6   month          897 non-null    object
 7   day            897 non-null    object
 8   regions        898 non-null    object
 9   tags           898 non-null    object
 10  summary        898 non-null    object
 11  og_link        898 non-null    object
 12  original_lang  898 non-null    object
 13  label          898 non-null    object
dtypes: object(14)
memory usage: 98.3+ KB


In [32]:
df_real_temp.isna().sum()

title            0
content          0
author           0
url              0
date             1
year             1
month            1
day              1
regions          0
tags             0
summary          0
og_link          0
original_lang    0
label            0
dtype: int64

In [33]:
df_real_temp.isnull().sum()

title            0
content          0
author           0
url              0
date             1
year             1
month            1
day              1
regions          0
tags             0
summary          0
og_link          0
original_lang    0
label            0
dtype: int64

In [43]:
df_real = df_real.append(df_real_temp,ignore_index = True)

C:\Users\annak\AppData\Local\Temp\ipykernel_8540\3604905186.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_real = df_real.append(df_real_temp,ignore_index = True)


In [18]:
df_real_temp.drop_duplicates()

,title,content,author,url,date,year,month,day,regions,tags,summary,og_link,original_lang,label
0,Ukraine war: What weapon killed 50 people in s...,A BBC investigation has found clear evidence t...,Manisha Ganguly and Joe Inwood,https://bbc.com/news/61079356,2022-04-13 00:00:00,2022,4,13,,,,https://bbc.com/news/61079356,en,0
1,Ukraine war student-turned-soldier: 'Friends d...,"Maxsym Lutsyk looks older and more serious, an...",Jeremy Bowen,https://bbc.com/news/world-europe-61624557,2022-05-30 00:00:00,2022,5,30,,,,https://bbc.com/news/world-europe-61624557,en,0
2,Roman Abramovich to sell Chelsea: 'It has not ...,,,https://bbc.com/sport/football/60597069,NA,0,0,0,,,,https://bbc.com/sport/football/60597069,en,0
3,Lindsey Graham: Russia issues arrest warrant f...,Russia has issued an arrest warrant for US Sen...,Matt Murphy,https://bbc.com/news/world-europe-65749077,2023-05-30 00:00:00,2023,5,30,,,,https://bbc.com/news/world-europe-65749077,en,0
4,Ukraine war: Russian pop megastar Alla Pugache...,"One of Russia's most popular singers, Alla Pug...",Laurence Peter,https://bbc.com/news/world-europe-62948146,2022-09-18 00:00:00,2022,9,18,,,,https://bbc.com/news/world-europe-62948146,en,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,Ukrainian drone destroys Russian supersonic bo...,A flagship Russian long-range bomber has been ...,Graeme Baker,https://bbc.com/news/world-europe-66573842,2023-08-22 00:00:00,2023,8,22,,,,https://bbc.com/news/world-europe-66573842,en,0
444,Niger coup: Is France to blame for instability...,Niger has become the latest country in West Af...,Leonard Mbulle-Nziege & Nic Cheeseman,https://bbc.com/news/world-africa-66406137,2023-08-06 00:00:00,2023,8,6,,,,https://bbc.com/news/world-africa-66406137,en,0
445,Tom Kerridge: Restaurant trade like 'walking u...,"Since Covid, there hasn't been any respite for...",Lucy Wallis,https://bbc.com/news/business-66360076,2023-08-03 00:00:00,2023,8,3,,,,https://bbc.com/news/business-66360076,en,0
446,Yemen: The children of a forgotten war,"If suffering had an address, it might be al-Ra...",Orla Guerin,https://bbc.com/news/world-middle-east-66258171,2023-07-25 00:00:00,2023,7,25,,,,https://bbc.com/news/world-middle-east-66258171,en,0


In [46]:
df_real.to_csv("real_bbc.csv" )

In [44]:
df_real

,title,content,author,url,date,year,month,day,regions,tags,summary,og_link,original_lang,label
0,"Ukraine war: Zelensky says 31,000 troops kille...","Volodymyr Zelensky says 31,000 Ukrainian soldi...",Kathryn Armstrong,https://bbc.com/news/world-europe-68397525,2024-02-25 00:00:00,2024.0,2.0,25.0,NaN,NaN,NaN,https://bbc.com/news/world-europe-68397525,en,0
1,All of Europe has 'skin in the game' in Ukrain...,,NaN,https://bbc.com/news/world-europe-68392036,NaN,0.0,0.0,0.0,NaN,NaN,NaN,https://bbc.com/news/world-europe-68392036,en,0
2,Ukrainian woman gives Welsh pub new lease of life,A woman who fled war-torn Ukraine hopes that r...,Andrew Arthur,https://bbc.com/news/articles/cj7vlvy1yvro,2024-03-05 00:00:00,2024.0,3.0,5.0,NaN,NaN,NaN,https://bbc.com/news/articles/cj7vlvy1yvro,en,0
3,Refugee returned to Kyiv due to state of UK home,A Ukrainian refugee returned to live in Kyiv r...,Gemma Dunstan,https://bbc.com/news/articles/c84jp1g2j71o,2024-03-14 00:00:00,2024.0,3.0,14.0,NaN,NaN,NaN,https://bbc.com/news/articles/c84jp1g2j71o,en,0
4,The unlikely species entangled in Ukraine's re...,"Lawsuits, art and the sandy blind mole-rat hav...",Darya Tsymbalyuk,https://bbc.com/future/article/20240223-ukrain...,2024-02-24 00:00:00,2024.0,2.0,24.0,NaN,NaN,NaN,https://bbc.com/future/article/20240223-ukrain...,en,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413,War in Ukraine: A family's struggle to get to ...,"A Scottish woman says she feels a ""sense of gu...",,https://bbc.com/news/uk-scotland-61192113,2022-04-24 00:00:00,2022,4,24,,,,https://bbc.com/news/uk-scotland-61192113,en,0
5414,Britons with Ukrainian family criticise visa b...,The difficulties faced by British families att...,,https://bbc.com/news/uk-england-northamptonshi...,2022-03-11 00:00:00,2022,3,11,,,,https://bbc.com/news/uk-england-northamptonshi...,en,0
5415,Oleksandr Zinchenko: Ukraine & Man City defend...,,,https://bbc.com/sport/av/football/60627422,NaN,NaN,NaN,NaN,,,,https://bbc.com/sport/av/football/60627422,en,0
5416,Shells rain down on Kharkiv as Ukraine's army ...,Ukraine's second city Kharkiv has been the con...,,https://bbc.com/news/world-europe-60785679,2022-03-17 00:00:00,2022,3,17,,,,https://bbc.com/news/world-europe-60785679,en,0
